In [1]:
# Compile deployment sheets

In [2]:
### import packages

import datetime
import metadataFunctions as mf
import numpy as np
import os
from os import path
import pandas as pd
import re
import subprocess
from subprocess import Popen


In [3]:
### Year-specific Cruise Info
year = '2023'
cruiseID = 'TN422'
instrumentStartTime = '2023-09-18T00:00:00'

In [4]:
### Define specific files
DeployListFile = 'cruises/' + year + '/Deploy' + year + '.csv'
MooringIDListFile = 'cruises/' + year + '/MooringID' + year + '.csv'
prelimDeployFile = 'cruises/' + year + '/deployments' + year + '_prelim.csv'


In [5]:
### setup directory pathways
amRepo = 'https://raw.githubusercontent.com/oceanobservatories/asset-management/master'

### load in github deployment sheets
CabledArray = pd.Series(['CE02SHBP','CE04OSBP','CE04OSPD','CE04OSPS','RS01SBPD','RS01SBPS',
                        'RS01SLBS','RS01SUM1','RS03AXBS','RS03AXPD','RS03AXPS','RS03INT2',
                        'RS03INT1','RS01SUM2','RS03CCAL','RS03ECAL','RS03ASHS'])
deploymentSheets = []
for array in CabledArray:
    deployFilePath = amRepo + '/deployment/' + array + '_Deploy.csv' 
    deploymentSheets.append(deployFilePath)
    
df_deploy = pd.concat([pd.read_csv(f, skip_blank_lines = True, comment='#') for f in deploymentSheets], ignore_index = True)

### Load in Deployment list
Deploy_list = pd.read_csv(DeployListFile)
Deploy_list = Deploy_list.fillna('UNKNOWN')

### Load in Mooring ID list
MooringID_list = pd.read_csv(MooringIDListFile)

# Deployment sheet Reference Designator key to startDate, AssetID, rawFile
df_deploy_sort = df_deploy.sort_values(by=["Reference Designator","startDateTime"],ascending=False)

RefDes_dict = {}
for i in df_deploy_sort['Reference Designator'].unique():
    RefDes_dict[i] = [{'deployDate':datetime.datetime.strptime(df_deploy_sort['startDateTime'][j], '%Y-%m-%dT%H:%M:%S'), 'deployEnd':df_deploy_sort['stopDateTime'][j],'deploymentNumber':df_deploy_sort['deploymentNumber'][j],'mooringID':df_deploy_sort['mooring.uid'][j],'lat':df_deploy_sort['lat'][j],'lon':df_deploy_sort['lon'][j],'deployDepth':df_deploy_sort['deployment_depth'][j],'waterDepth':df_deploy_sort['water_depth'][j],'AssetID':df_deploy_sort['sensor.uid'][j]} for j in df_deploy_sort[df_deploy_sort['Reference Designator']==i].index]

# MooringID node key to mooringID
MooringID_dict = MooringID_list.set_index('node').T.to_dict('series')


In [6]:
sparesList = []
referenceList = []
deploymentList = []

for index, row in Deploy_list.iterrows():
    RefDes = 'nan'
    leg = str(row['leg'])
    if 'SPARE' in row['assignment']:
        sparesList.append(row['instrumentType'] + ' ' + row['assetID'])
    elif 'REF' in row['assignment']:
        referenceList.append(row['instrumentType'] + ' ' + row['assetID'])
    else:
        node = row['assignment'].split('-')[0]
        deployFound = 0
        for RefKey in RefDes_dict:
            if row['instrumentType'] in RefKey and node in RefKey:
                RefDes = RefKey
                maxDeployDate = max(d['deployDate'] for d in RefDes_dict[RefKey])
                for deploy in RefDes_dict[RefKey]:
                    if deploy['deployDate'] == maxDeployDate:
                        # retrieve previous deployment number and increment by 1
                        deploymentNumber = int(deploy['deploymentNumber']) + 1
                        # retrieve previous lat, lon, deployment_depth, water_depth
                        lat = deploy['lat']
                        lon = deploy['lon']
                        deployDepth = deploy['deployDepth']
                        waterDepth = deploy['waterDepth']
                        mooringID = MooringID_dict[node]['mooringID']
                        nodeID = MooringID_dict[node]['nodeID']
                        deployFound = 1
                        notes = 'The following parameters are preliminary and will be verified and updated after deployment: mooringID; lat; lon; deployment_depth; water_depth'
                        paramList = [leg,cruiseID,'','','',RefDes,str(deploymentNumber),'1',instrumentStartTime,'',mooringID,nodeID,row['assetID'],str(lat),str(lon),'',str(deployDepth),str(waterDepth),notes,'','']
                        deployString = ",".join(paramList)
                        deploymentList.append(deployString)
        if deployFound == 0:
            print('no valid RefDes for ' + row['instrumentType'] + ' ' + node)
            print(row['assetID'])
            mooringID = MooringID_dict[node]['mooringID']
            nodeID = MooringID_dict[node]['nodeID']
            notes = 'The following parameters are preliminary and will be verified and updated after deployment: mooringID; lat; lon; deployment_depth; water_depth'
            paramList = [leg,cruiseID,'','','',row['instrumentType']+'-'+node,'1','1',instrumentStartTime,'',mooringID,nodeID,row['assetID'],'lat','lon','','deployDepth','waterDepth',notes,'','']
            deployString = ",".join(paramList)
            deploymentList.append(deployString)
        
print(sparesList)
print(referenceList)
print(deploymentList)

headerLine = ('leg,CUID_Deploy,deployedBy,CUID_Recover,recoveredBy,Reference Designator,deploymentNumber,versionNumber,startDateTime,stopDateTime,mooring.uid,node.uid,sensor.uid,lat,lon,orbit,deployment_depth,water_depth,notes,electrical.uid,assembly_template_revision\n')

with open(prelimDeployFile,'w') as f:
    f.write(headerLine)
    for entry in deploymentList:
        f.write('%s\n' % (entry))

no valid RefDes for HYDBBA110 LJ01C
PIRSN-HYDBBA-00001
no valid RefDes for TRHPHA302 MJ03C
ATAPL-67653-00002
no valid RefDes for ENGPRES SF01A
ATAPL-75664-00006
no valid RefDes for ENGPRES SF01B
ATAPL-75664-00002
no valid RefDes for ENGPRES SF03A
ATAPL-75664-00003
['ADCPS-I ATAPL-69825-00003', 'ADCPS-K ATAPL-58419-00001', 'ADCPT-B ATOSU-69826-00003', 'ADCPT-D ATAPL-58315-00001', 'ADCPT-E ATAPL-68073-00004', 'CAMPT-A UNKNOWN', 'CTDBP-N --', 'CTDBP-O ATOSU-69828-00003', 'CTDPF-A ATOSU-66662-00013', 'CTDPF-B ATCWK-67627-00009', 'CTDPF-L ATAPL-67977-00001', 'DOFST-A ATOSU-58694-00007', 'DOSTA-D (CTD) TBD', 'DOSTA-D (DP) UNKNOWN', 'ENGPRES ATAPL-75664-00005', 'FLCDR-A UNKNOWN', 'FLNTU-A UNKNOWN', 'FLOR-D ATAPL-58322-00001', 'FLOR-D ATAPL-58322-00007', 'HYDBB ATAPL-58324-00006', 'NUTNR-A ATAPL-68020-00004', 'OBSSP-A ATAPL-58331-00009', 'OPTAA-C ATAPL-69943-00004', 'OPTAA-D ATOSU-58332-00003', 'PARAD-B ATAPL-78452-00005', 'PC02W-A ATOSU-58336-00007', 'PHSEN-A ATOSU-58337-00008', 'PREST-A ATAP